In [ ]:
import logging
logging.basicConfig(level='ERROR')

In [ ]:
import pandas as pd

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U langchain
!pip install -q -U ctransformers[cuda]
!pip install chromadb
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!gdown 1Kr5nfwQ1hdg4VkKPJNmljnc4kdqTyo83 -O SHROOM_test-unlabeled.zip

Downloading...
From: https://drive.google.com/uc?id=1Kr5nfwQ1hdg4VkKPJNmljnc4kdqTyo83
To: /content/SHROOM_test-unlabeled.zip
100% 312k/312k [00:00<00:00, 4.27MB/s]


In [ ]:
!unzip /content/SHROOM_test-unlabeled.zip

Archive:  /content/SHROOM_test-unlabeled.zip
   creating: SHROOM_test-unlabeled/
  inflating: SHROOM_test-unlabeled/test.model-agnostic.json  
  inflating: SHROOM_test-unlabeled/test.model-aware.json  


01 : Running With LangChain

In [ ]:
import torch
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config, )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [ ]:
pipeline = pipeline(
        "text-generation",
        model=model_4bit,
        tokenizer=tokenizer,
        use_cache=True,
        device_map="auto",
        max_length=500,
        do_sample=True,
        top_k=5,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
)

In [48]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
llm = HuggingFacePipeline(pipeline=pipeline)

In [49]:
template_mt = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""
template_pg = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""
template_dm = """<s>[INST] You are a helpful, respectful and honest assistant.
Answer the following question using this json format: {{answer: (yes, no or maybe), score: (an integer number between 1 and 5, which 1 is for not supported and 5 is for full supported), description: (a description for your answer)}}.
{question} Is the Sentence supported by the Context above? [/INST] </s>
"""

In [50]:
def chat_with_llm(question , template):
    prompt = PromptTemplate(template=template, input_variables=["question"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    response = llm_chain.run({"question":question})
    return response

In [51]:
run_on_test = True # whether this baseline system is ran on the test splits or the val splits

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
import json
import regex

In [57]:
# change paths appropriately
# make sure the output filename is the same as the reference filename for the scoring program
path_val_model_aware = "/content/SHROOM_test-unlabeled/test.model-aware.json"
path_val_model_aware_output = "/content/drive/MyDrive/test.model-aware.json"

#from datasets import load_dataset
import json
import random
import numpy as np
import tqdm.notebook as tqdm
seed_val = 442
random.seed(seed_val)
np.random.seed(seed_val)

# simple JSON loading
with open(path_val_model_aware, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
print(num_sample)
#num_sample = 5
output_json = []
labels = ["Not Hallucination", "Hallucination"]
"""
SelfCheckGPT Usage: (LLM) Prompt
https://github.com/potsawee/selfcheckgpt
Context: {}
Sentence: {}
Is the sentence supported by the context above?
Answer Yes or No:
"""
pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}')
for i in tqdm.trange(num_sample):
    task = str(data_val_all[i]['task'])
    if run_on_test:
        # test splits will contain ids to ensure correct alignment before scoring
        id = int(data_val_all[i]['id'])
    hyp = str(data_val_all[i]['hyp'])
    src = str(data_val_all[i]['src'])
    tgt = str(data_val_all[i]['tgt'])

    if task == "PG":
        context = f"{src}"
    else: #i.e. task == "MT" or task == "DM":
        context = f"{tgt}"

    message = f"Context: {context}\nSentence: {hyp}\n"
    item_to_json=[]
    idx = 0
    for j in range(10):
      try:
        response = chat_with_llm(message, template_mt)
        answer = str(response.strip().lower()).replace("```", "")
        match = pattern.findall(answer)[0]
        print(match)
        p = json.loads(match)
        item_to_json.append(p)
        idx = idx + 1
        if (idx == 3):
          break;
      except:
        print("An exception occurred")

    if run_on_test:
        item_to_json.append({"id":id})

    output_json.append(item_to_json)

    f = open(path_val_model_aware_output, 'w', encoding='utf-8')
    json.dump(output_json, f)
    f.close()
print("done")

1500


  0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{
"answer": "yes",
"score": 5,
"description": "the sentence 'a crazy person' matches the context of a 'crazy person' or 'crackpot' or 'lunatic'."
}
{
"answer": "yes",
"score": 5,
"description": "the sentence 'a crazy person' matches the context of a 'crazy person' or 'crackpot' or 'lunatic'."
}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


...


{
  "answer": "yes",
  "score": 3,
  "description": "the sentence is partially supported by the context. the context states that it takes a giant rocket over 100 feet high to put a satellite or telescope in space, but the sentence does not mention the specific purpose of the rocket or the height of the rocket. therefore, the sentence is partially correct, but not entirely supported by the context."
}

{
  "answer": "yes",
  "score": 3,
  "description": "the sentence is partially supported by the context. the context states that it takes a giant rocket over 100 feet high to put a satellite or telescope in space, but the sentence does not mention the specific purpose of the rocket or the height of the rocket. therefore, the sentence is partially correct, but not entirely supported by the context."
}


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{
"answer": "maybe",
"score": 4,
"description": "the context mentions the use of a giant rocket to reach a height of 100 feet, but it does not specify whether this is used to launch a satellite or telescope into space. the sentence provided uses a similar phrase to describe the same concept. therefore, while the context and sentence are related concepts, they are not directly supported by each other."
}
the context states that a giant rocket can reach a height of 100 feet, but it does not specify if it is used to launch a satellite or telescope. the sentence provided describes a similar concept of using a giant rocket to launch a satellite or telescope into space. however, the context and sentence are not directly supported by each other as there is no clear connection between the rocket's height and its use to launch a satellite or telescope.
{
"answer": "maybe",
"score": 4,
"description": "the context mentions the use of a giant rocket to reach a height of 100 feet, but it does not s

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1123: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


{
"answer": "no",
"score": 1,
"description": "while the giant rocket is a common method for launching satellites and telescopes into space, it is not the only method available. other options such as ballistic missiles, space planes, and even space elevators are also considered for space launches, depending on the specific mission requirements."
}
{
"answer": "no",
"score": 1,
"description": "while the giant rocket is a common method for launching satellites and telescopes into space, it is not the only method available. other options such as ballistic missiles, space planes, and even space elevators are also considered for space launches, depending on the specific mission requirements."
}
done
